In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64
import re

from animalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GSLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Center(html.A(html.Img(height=150,width=150,src='data:image/png;base64,{}'.format(encoded_image.decode())), 
           href="http://www.snhu.edu",id='Customer logo',title='Grazioso Salvare')),
    html.H4("A Dashboard by Greg Roberge"),
    html.Div([
        dcc.Dropdown(
            id='animal-type-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'WRescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MWRescue'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRescue'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
        ),
    ]),
    html.Hr(),
    dt.DataTable(
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
        },
        style_cell={
            'textOverflow': 'ellipsis',
            'maxWidth': 0
        },
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True, "hideable": True,} for i in df.columns
        ],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_current=0,
        page_size=10,    
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(                
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Callback for the filter buttons
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
    [Input('animal-type-dropdown', 'value')])
def update_dashboard(value):
    if (value == "WRescue"):
        df = pd.DataFrame(shelter.read({"animal_type":"Dog",
                                        "breed":{"$in":[re.compile(".*Labrador Retriever Mix.*"),re.compile(".*Chesa Bay Retr.*"),re.compile(".*Newfoundland.*")]},
                                       "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}}))
    elif (value == "MWRescue"):
        df = pd.DataFrame(shelter.read({"animal_type":"Dog", 
                                        "breed":{"$in":[re.compile(".*German Shepherd.*"),re.compile(".*Alaskan Malamute.*"),
                                                        re.compile(".*Old English Sheepdog.*"),re.compile(".*Siberian Husky.*"),
                                                       re.compile(".*Rottweiler.*")]}, "sex_upon_outcome":"Intact Male",
                                       "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}}))
    elif (value == "DRescue"):
        df = pd.DataFrame(shelter.read({"animal_type": "Dog",
                                        "breed":{"$in":[re.compile(".*Doberman Pinsch.*"),re.compile(".*German Shepherd.*"),
                                                        re.compile(".*Golden Retriever.*"),re.compile(".*Bloodhound.*"),
                                                       re.compile(".*Rottweiler.*")]}, "sex_upon_outcome":"Intact Male",
                                       "age_upon_outcome_in_weeks":{"$gte":20,"$lte":300}}))
    else:
        df = pd.DataFrame(shelter.read({}))

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)

# Callback for highlighting selected cells
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback for pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed')
        )
    ]

# Callback for map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]



app